In [1]:
import pandas as pd

messages = pd.read_csv('SMSSpamCollection.txt', sep='\t',
                           names=["label", "message"])

In [2]:
messages

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [3]:
messages.shape

(5572, 2)

In [4]:
print(len(messages[messages['label']=='ham']))
print(len(messages[messages['label']=='spam']))

4825
747


In [5]:
print(messages['label'].isnull().sum())
print(messages['message'].isnull().sum())

0
0


In [6]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import re
from nltk.corpus import stopwords
ps=WordNetLemmatizer()

[nltk_data] Downloading package wordnet to C:\Users\Kushal
[nltk_data]     Garg\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
corpus=[]
for i in range (0,len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
    review = review.lower()
    review = review.split()
    review = [ps.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(corpus).toarray()

In [9]:
X.shape

(5572, 7098)

In [10]:
y=pd.get_dummies(messages['label'])
y=y.iloc[:,1].values


# Train Test Split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

# Training model using Naive bayes classifier

from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

y_pred=spam_detect_model.predict(X_test)

In [11]:
y_pred

array([0, 1, 0, ..., 0, 1, 0], dtype=uint8)

In [12]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)

cm

array([[936,  19],
       [  7, 153]], dtype=int64)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(corpus).toarray()

In [14]:
y=pd.get_dummies(messages['label'])
y=y.iloc[:,1].values


# Train Test Split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

# Training model using Naive bayes classifier

from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

y_pred=spam_detect_model.predict(X_test)

In [15]:

cm=confusion_matrix(y_test,y_pred)

cm

array([[955,   0],
       [ 31, 129]], dtype=int64)

In [16]:
import pickle
pickle.dump(spam_detect_model, open('model.pkl','wb'))
model = pickle.load(open('model.pkl','rb'))


In [17]:
message1=messages['message'][0]

In [18]:
data1=[message1]
vect=tfidf.transform(data1).toarray()
my_prediction=spam_detect_model.predict(vect)
my_prediction

array([0], dtype=uint8)

In [19]:
message1=messages['message'][2]
data1=[message1]
vect=tfidf.transform(data1).toarray()
my_prediction=spam_detect_model.predict(vect)
my_prediction[0]

1

In [20]:
message1

"Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"

In [21]:
pickle.dump(tfidf, open('transform.pkl','wb'))
transform = pickle.load(open('transform.pkl','rb'))